In [ ]:
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import keras
from keras_unet.models import custom_unet

In [ ]:
experiment = "test3" 

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
######################################################################################################
#experiment = "test3" 
######################################################################################################
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################
machine = f"{working_dir}/{experiment}/pre_processing/machine"
npy = f"{working_dir}/{experiment}/pre_processing/npy"

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy1",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks",
    "op": f"{working_dir}/{experiment}/post_processing/original_prediction",
    "pp": f"{working_dir}/{experiment}/post_processing/processed_prediction"
    
}


In [ ]:
npy = f"{working_dir}/{experiment}/pre_processing/npy1"
new_unet = f"{working_dir}/{experiment}/unet_prediction_save"

In [ ]:
import keras
from keras_unet.models import custom_unet

In [ ]:
def dataset1():
    imagedata = np.load(os.path.join(npy, 'poo.npz'))
    labeldata = np.load(os.path.join(npy, 'pom.npz'))
    return imagedata,labeldata
    

In [ ]:
def dataset2():
    imagedata = np.load(os.path.join(npy, 'original.npz'))
    labeldata = np.load(os.path.join(npy, 'mask.npz'))
    return imagedata,labeldata

In [ ]:
# imagedata = np.load(os.path.join(npy, 'original.npz'))
# labeldata = np.load(os.path.join(npy, 'mask.npz'))

In [ ]:
imagedata = np.load(os.path.join(npy, 'poo.npz'))
labeldata = np.load(os.path.join(npy, 'pom.npz'))

In [ ]:
images_file = imagedata['data']
images_name = imagedata['names']

In [ ]:
len (images_name)


In [ ]:
labels_file = labeldata['data']
labels_name = labeldata['names']

In [ ]:
images = images_file
labels = labels_file

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
 # full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)
#test = test.reshape(test.shape[0],test.shape[1],test.shape[2],1)

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
p = np.random.permutation(len(images))

In [ ]:
images = images[p]
labels = labels[p]

In [ ]:
labels = labels.astype(np.float64)

In [ ]:
# images = images.astype(np.float64)
# for i in range(images.shape[0]):
#     images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets with a ratio of 85:15
# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.15)

# # Further split the training set into training and validation sets with a ratio of 82:18
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765)

In [ ]:
X_train = images[0:1350]
y_train = labels[0:1350]
X_val = images[1351:1570]
y_val = labels[1351:1570]
X_test = images[1571:]
y_test = labels[1571:]

In [ ]:
y_train.shape

In [ ]:
X_train.shape

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs
import cv2
from PIL import Image
from skimage.filters import threshold_otsu
from keras.preprocessing.image import ImageDataGenerator

import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
model = custom_unet(
                    input_shape=(512, 512, 1),
                    use_batch_norm=False,
                    num_classes=1,
                    filters=32,
                    dropout=0.5,
                    output_activation='sigmoid')

In [ ]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [ ]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
path = '/raid/mpsych/RISTERLAB/kiran/flyem/K_Experiments/models/Unet'

In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# Training with early stopping
history = model.fit(X_train,
                    y_train,
                    batch_size=5,
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])


y_pred = model.predict(X_test)

for i in range(15):
    # Apply thresholding
    threshold = 0.2
    y_pred_binary = (y_pred [i]> threshold).astype(np.uint8)
    binary_mask = y_pred_binary
    z = images_name[i]
    plt.imshow(binary_mask)
    plt.show()

# save the updated weights of the new model
model.save_weights(f'{path}/new_Uidmodel1.h5')
print (f'saved model : new_Umodel1.h5')
threshold = 0.2
y_pred_binary = (y_pred > threshold).astype(np.uint8)
plot_segm_history(history)   
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)
    # Convert the binary_mask variable to a 2D array
  #  binary_mask_2d = binary_mask.squeeze()
    # Convert the binary_mask variable to uint8 format
#    binary_mask_uint8 = (binary_mask_2d * 255).astype(np.uint8)
   # save_path = os.path.join(new_unet, z) 
   # binary_mask_new = np.array(binary_mask_uint8)
    #mh.imsave(save_path, binary_mask_new)
    
 # Plotting results
# Apply thresholding


# history.history.keys()
# metrics = list(history.history.keys())
# plot_segm_history(history, metrics=metrics)   
# plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=30)

# #print ('completed iteration number',k,'The value of al is ',al)
print ('#############################')
print()
print()
print()

In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# Training with early stopping
history = model.fit(X_train,
                    y_train,
                    batch_size=60,
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])


y_pred = model.predict(X_test)

for i in range(15):
    # Apply thresholding
    threshold = 0.2
    y_pred_binary = (y_pred > threshold).astype(np.uint8)
    binary_mask = y_pred_binary[i]
    z = images_name[i]
    # Convert the binary_mask variable to a 2D array
    binary_mask_2d = binary_mask.squeeze()
    # Convert the binary_mask variable to uint8 format
    binary_mask_uint8 = (binary_mask_2d * 255).astype(np.uint8)
    save_path = os.path.join(new_unet, z) 
    binary_mask_new = np.array(binary_mask_uint8)
    mh.imsave(save_path, binary_mask_new)
    
# save the updated weights of the new model
new_model.save_weights(f'{path}/new_Umodel{bl}.h5')
print (f'saved model : new_Umodel{bl}.h5')
threshold = 0.2
y_pred_binary = (y_pred > threshold).astype(np.uint8)
plot_segm_history(history)   
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

# history.history.keys()
# metrics = list(history.history.keys())
# plot_segm_history(history, metrics=metrics)   
# plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=30)

# #print ('completed iteration number',k,'The value of al is ',al)
print ('#############################')
print()
print()
print()

In [ ]:
def create_Umodel():
    import keras.optimizers
    from keras_unet.metrics import iou, iou_thresholded
    from keras_unet.losses import jaccard_distance
    model = custom_unet(
                    input_shape=(512, 512, 1),
                    use_batch_norm=False,
                    num_classes=1,
                    filters=32,
                    dropout=0.5,
                    output_activation='sigmoid')


In [ ]:
def wtrain(number):
    from keras import regularizers
    from keras.layers import Dense
    from keras.callbacks import EarlyStopping
    from keras.preprocessing.image import ImageDataGenerator
    from keras_unet.utils import plot_segm_history, plot_imgs
    import cv2
    from PIL import Image
    from skimage.filters import threshold_otsu
    from keras.preprocessing.image import ImageDataGenerator
    al = 1
    for k in range(number):
        # create a new model with the same architecture as the original model
        new_model = create_Umodel()
        path = '/raid/mpsych/RISTERLAB/kiran/flyem/K_Experiments/models/Unet'
        # load the previously saved weights into the new model
        new_model.load_weights(f'{path}/new_Umodel{al}.h5')
        print (f'loaded model : new_Umodel{al}.h5')
        # compile the new model        
        opt = keras.optimizer_v1.Adam(lr=0.01)
        # compile the new model
        new_model.compile(optimizer = 'Adam',    
                          loss='binary_crossentropy', 
                          metrics=[iou, iou_thresholded])
        # Early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        # Training with early stopping
        history = model.fit(X_train,
                            y_train,
                            batch_size=60,
                            epochs=700,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping])


        bl = (al+1)
        # save the updated weights of the new model
        new_model.save_weights(f'{path}/new_Umodel{bl}.h5')
        
        print (f'saved model : new_Umodel{bl}.h5')
        # Make predictions and apply thresholding
        y_pred = new_model.predict(X_test)



#         for i in range(5):
#             binary_mask = y_pred[i]
#             z = images_name[i]
#             threshold = 0.2
#             binary_mask = (binary_mask[i] > threshold).astype(np.uint8)
#             # Convert the binary_mask variable to a 2D array
#             binary_mask_2d = binary_mask.squeeze()
#             Convert the binary_mask variable to uint8 format
#             binary_mask_uint8 = (binary_mask_2d * 255).astype(np.uint8)
#             save_path = os.path.join(dir_path, z) 
#             binary_mask_new = np.array(binary_mask_uint8)
#             mh.imsave(save_path, binary_mask_new)
            
        threshold = 0.2
        y_pred_binary = (y_pred[i] > threshold).astype(np.uint8)
        plot_segm_history(history)   
        plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10
        al += 1
        print ('completed iteration number',k,'The value of al is ',al)
        print ('#############################')
        print()
        print()
        print()

In [ ]:
wtrain(5)